## Comparing New York to Toronto to determine how similiar or dissimilar  they are
Using the neighborhood data for NY and TO, I will build K amount
of clusters based on their top ten venues. If it turns out that most clusters do
not include TO and NY neighborhoods, we can say TO and NY are dissimilar 

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in New York City</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

## 1. Download Toronto City into Dataframe TO_neighborhoods

In [ ]:
import pandas as pd # library for data analsysis
import requests # library to handle requests


In [ ]:
url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
htmltext = url.text
print('XML Downloaded')

In [ ]:
from bs4 import BeautifulSoup
import bs4

soup = bs4.BeautifulSoup(htmltext, 'lxml')
table = soup.find('table','wikitable sortable')
#print (table)

In [ ]:
            n_columns = 0
            n_rows=0
            column_names = []
    
            # Find number of rows and columns
            # we also find the column titles if we can
            for row in table.find_all('tr'):
                
                # Determine the number of rows in the table
                td_tags = row.find_all('td')
                if len(td_tags) > 0:
                    n_rows+=1
                    if n_columns == 0:
                        # Set the number of columns for our table
                        n_columns = len(td_tags)
                        
                # Handle column names if we find them
                th_tags = row.find_all('th') 
                if len(th_tags) > 0 and len(column_names) == 0:
                    for th in th_tags:
                        column_names.append(th.get_text().replace('\n', ''))
    
            # Safeguard on Column Titles
            if len(column_names) > 0 and len(column_names) != n_columns:
                raise Exception("Column titles do not match the number of columns")
            print (column_names)
            columns = column_names if len(column_names) > 0 else range(0,n_columns)
            df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
            row_marker = 0
            for row in table.find_all('tr'):
                column_marker = 0
                columns = row.find_all('td')
                for column in columns:
                    df.iat[row_marker,column_marker] = column.get_text()
                    column_marker += 1
                if len(columns) > 0:
                    row_marker += 1
                    
df = df.replace('\n','', regex=True)
df = df[df.Borough != "Not assigned"]
df.head()

In [ ]:
!wget -O Geospatial_data.csv http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
    
    

In [ ]:
Geospatial_data_df = pd.read_csv("Geospatial_data.csv")

Geospatial_data_df.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
TO_neighborhoods = pd.merge(df, Geospatial_data_df, on='Postcode')

TO_neighborhoods.rename(columns={'Postcode': 'City'}, inplace=True)
TO_neighborhoods.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=True)
TO_neighborhoods['City'] = 'TO'
TO_neighborhoods.head()

downtown_toronto_data = TO_neighborhoods[TO_neighborhoods['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_toronto_data['Borough'] = 'TO_' + downtown_toronto_data['Borough']
downtown_toronto_data['Neighborhood'] = 'TO_' + downtown_toronto_data['Neighborhood'] 
downtown_toronto_data.head()

<a id='item1'></a>

## 1. Download New York City into Dataframe NY_neighborhoods

In [ ]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


In [ ]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset

with open('newyork_data.json') as NY_json_data:
    NY_data = json.load(NY_json_data)
    
# Notice how all the relevant data is in the features key, which is basically a list of the neighborhoods. 
# So, let's define a new variable that includes this data   

NY_features_data = NY_data['features']

# Tranform the JSON data into a pandas dataframe
# define the dataframe columns
column_names = ['City', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
NY_neighborhoods = pd.DataFrame(columns=column_names)

for data in NY_features_data:
    city = 'NY'
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    NY_neighborhoods = NY_neighborhoods.append({'City': city,
                                          'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

print ("NY Loaded")

In [ ]:
NY_neighborhoods.head()
NY_neighborhoods.shape

In [ ]:
manhattan_data = NY_neighborhoods[NY_neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)

In [ ]:
manhattan_data['Borough'] = 'NY_' + manhattan_data['Borough'] 
manhattan_data['Neighborhood'] = 'NY_' + manhattan_data['Neighborhood'] 
manhattan_data.head()


## 1. Concatenate the NY neighborhoods with Toronto neighborhoods into neighborhoods

In [ ]:
TO_neighborhoods.reset_index()
neighborhood_data = pd.concat([manhattan_data, downtown_toronto_data], axis=0).reset_index()
neighborhood_data.head()

In [ ]:
neighborhood_data.tail()

In [ ]:
neighborhood_data.shape

#### Define Foursquare Credentials and Version

In [ ]:
CLIENT_ID = 'JVFWKWHDS0TWEDRK4F44GHOHWOS1DOHFZLRI14J1BKB5YO12' # your Foursquare ID
CLIENT_SECRET = 'HFR4PME5VUMUESCRPCDIGP2OBHC2HI4TF4ZYHEFZM3NA0FHP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Now we are ready to clean the json and structure it into a *pandas* dataframe.

<a id='item2'></a>

In [ ]:
## Explore the neighborhoods or NY and TO

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [ ]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

neighborhood_venues = getNearbyVenues(names=neighborhood_data['Neighborhood'],
                                   latitudes=neighborhood_data['Latitude'],
                                   longitudes=neighborhood_data['Longitude']
                                  )



#### Let's check the size of the resulting dataframe

In [ ]:
print(neighborhood_venues.shape)
neighborhood_venues.head(100)

Let's check how many venues were returned for each neighborhood

In [ ]:
neighborhood_venues.groupby('Neighborhood').count()

#### Let's find out how many unique categories can be curated from all the returned venues

In [ ]:
print('There are {} uniques categories.'.format(len(neighborhood_venues['Venue Category'].unique())))

<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [ ]:
# one hot encoding
neighborhood_onehot = pd.get_dummies(neighborhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
neighborhood_onehot['Neighborhood'] = neighborhood_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [neighborhood_onehot.columns[-1]] + list(neighborhood_onehot.columns[:-1])
neighborhood_onehot = neighborhood_onehot[fixed_columns]

neighborhood_onehot.head()

And let's examine the new dataframe size.

In [ ]:
neighborhood_onehot.shape

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
neighborhood_grouped = neighborhood_onehot.groupby('Neighborhood').mean().reset_index()
neighborhood_grouped

#### Let's confirm the new size

In [ ]:
neighborhood_grouped.shape

#### Let's print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in neighborhood_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = neighborhood_grouped[neighborhood_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [ ]:
import numpy as np # library to handle data in a vectorized manner

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = neighborhood_grouped['Neighborhood']

for ind in np.arange(neighborhood_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neighborhood_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [ ]:

# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 8

neighborhood_grouped_clustering = neighborhood_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neighborhood_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

neighborhood_merged = neighborhood_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
neighborhood_merged = neighborhood_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

neighborhood_merged.head() # check the last columns!

Finally, let's visualize the resulting clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [ ]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 0, neighborhood_merged.columns[[3] + list(range(5, neighborhood_merged.shape[1]))]]

In [ ]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 1, neighborhood_merged.columns[[1] + list(range(5, neighborhood_merged.shape[1]))]]

In [ ]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 2, neighborhood_merged.columns[[1] + list(range(5, neighborhood_merged.shape[1]))]]

In [ ]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 3, neighborhood_merged.columns[[1] + list(range(5, neighborhood_merged.shape[1]))]]

In [ ]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 4, neighborhood_merged.columns[[1] + list(range(5, neighborhood_merged.shape[1]))]]

In [ ]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 5, neighborhood_merged.columns[[1] + list(range(5, neighborhood_merged.shape[1]))]]

In [ ]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 6, neighborhood_merged.columns[[1] + list(range(5, neighborhood_merged.shape[1]))]]

In [ ]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 7, neighborhood_merged.columns[[1] + list(range(5, neighborhood_merged.shape[1]))]]